In [16]:
import numpy as np

DATA_FOLDER = './data/'
NUM_INSTANCE = 4367
NUM_OBJ = 111
NUM_STATE = 2

data_file = open(DATA_FOLDER + 'chowliu-input.txt', 'rb')

D = np.zeros((NUM_INSTANCE, NUM_OBJ)) # data matrix
for i, line in enumerate(data_file.readlines()):
    line = line.replace('\n', '')
    line = line.split()
    assert len(line) == NUM_OBJ
    for j, w in enumerate(line):
        D[i][j] = float(w)

data_file.close()

In [17]:
np.shape(D)

(4367, 111)

In [18]:
P_single = np.zeros((NUM_OBJ, NUM_STATE))
P_single[:, 1] = np.sum(D, axis=0) / NUM_INSTANCE
P_single[:, 0] = 1 - P_single[:, 1]

In [19]:
print(P_single[0])
print(P_single[2])

[0.99175635 0.00824365]
[0.97939089 0.02060911]


In [20]:
def mutual_info(joint_dist, u, v):
    res = 0.0
    n = len(joint_dist)
    for i in range(n):
        for j in range(n):
            if joint_dist[i][j] > 0.0:
                res += joint_dist[i][j] * (np.log(joint_dist[i][j]) - 
                        np.log(P_single[u][i]) - np.log(P_single[v][j]))
    return res

In [21]:
A = np.zeros((NUM_OBJ, NUM_OBJ)) # adjacent matrix
for i in range(NUM_OBJ):
    for j in range(NUM_OBJ - i - 1):
        k = j + i + 1

        # process A[i][k]      
        cnt = np.zeros((NUM_STATE, NUM_STATE))
        for sample in range(NUM_INSTANCE):
            cnt[int(D[sample][i])][int(D[sample][k])] += 1

        assert np.sum(cnt) == NUM_INSTANCE
        
        # joint distribution for X_i, X_k
        cnt /= NUM_INSTANCE

        A[k][i] = A[i][k] = mutual_info(cnt, i, k)

In [22]:
print(A[0][2])
print(A[2][0])

0.0001723883503887336
0.0001723883503887336


In [23]:
# prim - minimum spanning tree
# Tim Wilson, 2-25-2002

#A = adjacency matrix, u = vertex u, v = vertex v
def weight(A, u, v):
    return A[u][v]

#A = adjacency matrix, u = vertex u
def adjacent(A, u):
    L = []
    for x in range(len(A)):
        if A[u][x] > 0 and x <> u:
            L.insert(0,x)
    return L

#Q = max queue
def extractMax(Q):
    q = Q[0]
    Q.remove(Q[0])
    return q

#Q = max queue, V = vertex list
def increaseKey(Q, K):
    for i in range(len(Q)):
        for j in range(len(Q)):
            if K[Q[i]] > K[Q[j]]:
                s = Q[i]
                Q[i] = Q[j]
                Q[j] = s

#V = vertex list, A = adjacency list, r = root
def prim(V, A, r):
    u = 0
    v = 0

    # initialize and set each value of the array P (pi) to none
    # pi holds the parent of u, so P(v)=u means u is the parent of v
    P=[None]*len(V)

    # initialize and set each value of the array K (key) to zero
    K = [0]*len(V)

    # initialize the max queue and fill it with all vertices in V
    Q=[0]*len(V)
    for u in range(len(Q)):
        Q[u] = V[u]

    # set the key of the root to 0
    K[r] = 9999999
    increaseKey(Q, K)    # maintain the max queue

    # loop while the min queue is not empty
    while len(Q) > 0:
        u = extractMax(Q)    # pop the first vertex off the max queue

        # loop through the vertices adjacent to u
        Adj = adjacent(A, u)
        for v in Adj:
            w = weight(A, u, v)    # get the weight of the edge uv

            # proceed if v is in Q and the weight of uv is greater than v's key
            if Q.count(v) > 0 and w > K[v]:
                # set v's parent to u
                P[v] = u
                # v's key to the weight of uv
                K[v] = w
                increaseKey(Q, K)    # maintain the min queue
    return P

In [24]:
V = range(NUM_OBJ)
P = prim(V, A, 85)

In [25]:
P

[72,
 46,
 20,
 85,
 20,
 85,
 40,
 85,
 108,
 66,
 78,
 13,
 13,
 46,
 85,
 14,
 22,
 80,
 17,
 82,
 46,
 72,
 32,
 47,
 72,
 24,
 46,
 46,
 36,
 85,
 85,
 85,
 84,
 93,
 108,
 8,
 26,
 65,
 93,
 20,
 110,
 22,
 108,
 102,
 85,
 1,
 85,
 106,
 95,
 85,
 102,
 46,
 40,
 50,
 0,
 93,
 108,
 85,
 85,
 93,
 50,
 88,
 108,
 8,
 102,
 16,
 83,
 72,
 64,
 40,
 33,
 46,
 20,
 74,
 108,
 46,
 78,
 36,
 46,
 108,
 85,
 85,
 85,
 72,
 108,
 None,
 1,
 52,
 85,
 20,
 46,
 90,
 46,
 84,
 72,
 26,
 46,
 24,
 84,
 84,
 20,
 14,
 46,
 72,
 20,
 24,
 46,
 80,
 46,
 46,
 58]